In [1]:
from graphdatascience import GraphDataScience
from getpass import getpass
import pandas as pd

/Users/nathansmith/anaconda3/envs/chatbot/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Set up connections

In [3]:
vertex_ai_token = getpass()

 ········


In [4]:
neo4j_password = getpass()

 ········


In [9]:
neo4j_uri = "neo4j+s://bc9f751a.databases.neo4j.io"
neo4j_user = "neo4j"
gds = GraphDataScience(neo4j_uri, auth=(neo4j_user, neo4j_password))

# Embed summaries using same model as documents

In [7]:
gds.run_cypher("""
CREATE VECTOR INDEX generated_question_embeddings IF NOT EXISTS 
                  FOR (g:GeneratedQuestion)
                  ON (g.embedding)
                  OPTIONS {indexConfig: 
                      {`vector.dimensions`: 768,
                       `vector.similarity_function`: 'cosine'
                       }}
                       """)

""


In [10]:
gds.run_cypher("""
MATCH (g:GeneratedQuestion)
CALL {
WITH g
WITH collect(g) AS questionNodes, collect(g.content) AS content
CALL genai.vector.encodeBatch(content, "VertexAI", { token: $token, projectId: $project }) 
YIELD index, vector
WITH questionNodes[index] AS g, vector
CALL db.create.setNodeVectorProperty(g, 'embedding', vector)
} IN TRANSACTIONS Of 1000 ROWS""",
              {"token": vertex_ai_token, "project": "neo4j-ps-202001"})


""
